# Introduction

Green-Plant Management GmbH is a company specializing in vegetation management
solutions, that primarily uses optical satellite imagery. Currently, they are focused on a
government project aimed at monitoring deforestation in the Amazonas region via time-series
analysis. To accomplish this, they want to use UP42 and Sentinel-2 images.
They are concerned about the availability of satellite images within their designated area of
interest (AOI). They want to construct a tailored pipeline using the UP42 API or UP42 Python
SDK, thereby automating the entire process.

### Task Overview
- Objective: Develop a Jupyter notebook for Green-Plant Management GmbH, focusing on monitoring deforestation in the Amazonas region using time-series analysis of satellite imagery.
- Tools & Technologies: UP42 API or UP42 Python SDK, Rasterio, Numpy, Geopandas, and SpatioTemporal Asset Catalog (STAC).
- Key Focus: Use NDVI (Normalized Difference Vegetation Index) for the time-series analysis of a specific area of interest (AOI) within the Amazonas region.

In [3]:
import up42
from getpass import getpass

user_email = input("Please enter your email: ")
pw = getpass()

up42.authenticate(
    username= user_email,
    password= pw,
)

2023-11-27 12:32:16,519 - Authentication with UP42 successful!


## Create the AOI
Since the customer's AOI was not provided, we will create a small script to manually generate the AOI visually picking it out using satellite imagery.

In [1]:
from ipyleaflet import Map, DrawControl, TileLayer
import geojson

# Declare the the location of where the AOI will be drawn and create the map
aoi_longlat = (-9.383746, -67.494271)
m = Map(center=aoi_longlat, zoom=13)

# Create the satellite layer by pulling it from Esri and add it to the map as a layer
satellite_layer = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attribution="Sources: Esri, Maxar, GeoEye, Earthstar Geographics, CNES/Airbus DS, USDA, USGS, AeroGRID, IGN, and the GIS User Community"
)
m.add_layer(satellite_layer)

# Create the draw control and add it to the map
draw_control = DrawControl()
m.add_control(draw_control)

# Create the drawn geojson as an empty variable which can then be written to
drawn_geojson = None

# Create function to tell the draw control to write the drawn geojson to the variable (globally)
def handle_draw(target, action, geo_json):
    global drawn_geojson
    drawn_geojson = geojson.dumps(geo_json)

draw_control.on_draw(handle_draw)

display(m)

Map(center=[-9.383746, -67.494271], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [2]:
# Write the drawn geojson to a file
with open('amazonas_aoi.geojson', 'w') as f:
    f.write(drawn_geojson)

## Access the catalog and add the relevant imagery to storage on UP42

Since we don't know what kind of data is available and how good they are, we want to search through the catalog using our created AOI and find appropriate data for this project.

### Access the catalog

Here we will get the list of available data products. Although we know we will use Sentinel-2, we still want to get the relevant IDs and see how the catalog is structured

In [5]:
from pprint import pprint

catalog = up42.initialize_catalog()
products = catalog.get_data_products(basic=True)

#Pretty print the catalog for readability
pprint(products)

{'Beijing-3A': {'collection': 'beijing-3a',
                'data_products': {'Level 1 (mono)': 'e080274b-6cfb-4a89-bc12-64fd4895b3c2',
                                  'Level 1 (stereo)': 'ab6fbe7d-2eb6-4f84-8337-418db71d8e75'},
                'host': '21at'},
 'Capella Space GEC': {'collection': 'capella-gec',
                       'data_products': {'Full Scene': '96072809-d820-4cf9-86dd-d3bff3337c35'},
                       'host': 'capellaspace'},
 'Capella Space GEO': {'collection': 'capella-geo',
                       'data_products': {'Full Scene': 'd66facaa-533f-49a2-849a-c2910ac9dd31'},
                       'host': 'capellaspace'},
 'Capella Space SICD': {'collection': 'capella-sicd',
                        'data_products': {'Full Scene': '8b0aed07-c565-4bf9-b719-401e692de4a6'},
                        'host': 'capellaspace'},
 'Capella Space SLC': {'collection': 'capella-slc',
                       'data_products': {'Full Scene': '1f2b0d7f-d3e2-4b3d-96b7-e7c184df7952

### Searching the sentinl-2 collection for relevant data
Now that we got the collection ID and the AOI, we can input them into our search parameters to find the most relevant data

In [31]:
#Read the created AOI geojson
geom = up42.read_vector_file("amazonas_aoi.geojson")

#Create the search parameters
search_parameters = catalog.construct_search_parameters(
    collections = ["sentinel-2"],
    geometry = geom,
    start_date = "2018-01-01",
    end_date = "2023-12-31",
    max_cloudcover = 1,
    limit = 200,
)

search_results = catalog.search(search_parameters)

2023-11-27 15:22:16,234 - Searching catalog with search_parameters: {'datetime': '2018-01-01T00:00:00Z/2023-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((-67.575703, -9.402492), (-67.578106, -9.444151), (-67.528324, -9.451263), (-67.521629, -9.400291), (-67.575703, -9.402492)),)}, 'limit': 200, 'collections': ['sentinel-2'], 'query': {'cloudCoverage': {'lte': 1}}}


2023-11-27 15:22:18,331 - 52 results returned.


### Inspecting the search results

There seems to a good number of results with very low cloud coverage. From a simple visual inspection, we can see that there is data available every year from 2018 to 2023 in late July with very low coverage.
In this case, we can opt to pick one scene from each year between 2018 and 2023 from late July for our time series analysis

In [32]:
#Show the search results
search_results

,geometry,id,constellation,collection,providerName,up42:usageType,providerProperties,sceneId,producer,acquisitionDate,start_datetime,end_datetime,cloudCoverage,resolution,deliveryTime
0,"POLYGON ((-68.09011 -9.04508, -68.08747 -10.03...",S2A_19LFK_20230908_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2A_19LFK_20230908_0_L2A,european-space-agency,2023-09-08T14:55:37.264Z,None,None,0.047827,10.0,MINUTES
1,"POLYGON ((-68.09011 -9.04508, -68.08747 -10.03...",S2B_19LFK_20230725_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20230725_0_L2A,european-space-agency,2023-07-25T14:55:38.054Z,None,None,0.000345,10.0,MINUTES
2,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20230705_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20230705_0_L2A,european-space-agency,2023-07-05T14:55:37.486Z,None,None,0.001191,10.0,MINUTES
3,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20230625_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20230625_0_L2A,european-space-agency,2023-06-25T14:55:36.568Z,None,None,0.616796,10.0,MINUTES
4,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20221107_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20221107_0_L2A,european-space-agency,2022-11-07T14:55:30.734Z,None,None,0.000415,10.0,MINUTES
5,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2A_19LFK_20220903_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2A_19LFK_20220903_0_L2A,european-space-agency,2022-09-03T14:55:44.034Z,None,None,0.002677,10.0,MINUTES
6,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2A_19LFK_20220814_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2A_19LFK_20220814_0_L2A,european-space-agency,2022-08-14T14:55:44.473Z,None,None,0.280537,10.0,MINUTES
7,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20220730_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20220730_0_L2A,european-space-agency,2022-07-30T14:55:36.951Z,None,None,0.000760,10.0,MINUTES
8,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20220720_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20220720_0_L2A,european-space-agency,2022-07-20T14:55:36.986Z,None,None,0.000902,10.0,MINUTES
9,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20220710_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:datatake_type': 'INS-NOBS', 's2:degraded_...",S2B_19LFK_20220710_0_L2A,european-space-agency,2022-07-10T14:55:36.690Z,None,None,0.289196,10.0,MINUTES


### Filter the results according to our parameters and add place the order to add the assets to our storage

In [30]:
import pandas as pd

# Make sure the acquisition date is in pandas datetime format for easier filtering
search_results['acquisitionDate'] = pd.to_datetime(search_results['acquisitionDate'])

# Filter for July data between the dates of 25th and 31st
july_data = search_results[(search_results['acquisitionDate'].dt.month == 7) & (search_results['acquisitionDate'].dt.day >= 25) & (search_results['acquisitionDate'].dt.day <= 31)]

# Since we only need one scene each year we can just take the first record as the cloud coverage is all minimal
first_record_each_year = july_data.groupby(july_data['acquisitionDate'].dt.year).first()

#Display the results of our scenes of interest
first_record_each_year 



,geometry,id,constellation,collection,providerName,up42:usageType,providerProperties,sceneId,producer,acquisitionDate,start_datetime,end_datetime,cloudCoverage,resolution,deliveryTime
acquisitionDate,,,,,,,,,,,,,,,
2018,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2A_19LFK_20180726_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:degraded_msi_data_percentage': 0, 's2:dat...",S2A_19LFK_20180726_0_L2A,european-space-agency,2018-07-26 14:54:59.825000+00:00,None,None,0.029343,10.0,MINUTES
2019,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2A_19LFK_20190731_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:degraded_msi_data_percentage': 0, 's2:dat...",S2A_19LFK_20190731_0_L2A,european-space-agency,2019-07-31 14:55:36.740000+00:00,None,None,0.027737,10.0,MINUTES
2020,"POLYGON ((-68.09011 -9.04508, -68.08747 -10.03...",S2B_19LFK_20200730_1_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:degraded_msi_data_percentage': 0, 's2:dat...",S2B_19LFK_20200730_1_L2A,european-space-agency,2020-07-30 14:55:34.843000+00:00,None,None,0.466963,10.0,MINUTES
2021,"POLYGON ((-68.09011 -9.04508, -68.08747 -10.03...",S2A_19LFK_20210730_1_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:degraded_msi_data_percentage': 0.0001, 's...",S2A_19LFK_20210730_1_L2A,european-space-agency,2021-07-30 14:55:35.361000+00:00,None,None,0.000992,10.0,MINUTES
2022,"POLYGON ((-68.09011 -9.04508, -67.09137 -9.041...",S2B_19LFK_20220730_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:degraded_msi_data_percentage': 0, 's2:dat...",S2B_19LFK_20220730_0_L2A,european-space-agency,2022-07-30 14:55:36.951000+00:00,None,None,0.000760,10.0,MINUTES
2023,"POLYGON ((-68.09011 -9.04508, -68.08747 -10.03...",S2B_19LFK_20230725_0_L2A,sentinel-2,sentinel-2,earthsearch-aws,[DATA],"{'s2:degraded_msi_data_percentage': 0, 's2:dat...",S2B_19LFK_20230725_0_L2A,european-space-agency,2023-07-25 14:55:38.054000+00:00,None,None,0.000345,10.0,MINUTES


In [45]:
# With the product ID we got from searching the catalog we can now place an order for each of the corresponding scene of interest
for i in range(len(first_record_each_year)):
    order_parameters = catalog.construct_order_parameters(
        data_product_id = "c3de9ed8-f6e5-4bb5-a157-f6430ba756da",
        image_id = first_record_each_year.iloc[i]["id"],
        aoi = geom,
    )
    order = catalog.place_order(order_parameters)
    
    #order.track_status()
    

2023-11-27 15:54:19,355 - See `catalog.get_data_product_schema(data_product_id)` for more detail on the parameter options.
2023-11-27 15:54:26,888 - Order is PLACED
2023-11-27 15:54:26,889 - Order 283e1113-c297-4f66-92e7-61638e4ccf95 is now PLACED.
2023-11-27 15:54:26,890 - Tracking order status, reporting every 120 seconds...
2023-11-27 15:54:27,515 - Order is PLACED
2023-11-27 15:54:28,106 - Order is PLACED
2023-11-27 15:56:29,363 - Order is BEING_FULFILLED
2023-11-27 15:56:29,970 - Order is BEING_FULFILLED
2023-11-27 15:56:30,638 - Order is BEING_FULFILLED! - 283e1113-c297-4f66-92e7-61638e4ccf95
2023-11-27 15:56:30,639 - 
2023-11-27 15:58:31,282 - Order is BEING_FULFILLED
2023-11-27 15:58:31,881 - Order is BEING_FULFILLED
2023-11-27 15:58:32,487 - Order is BEING_FULFILLED! - 283e1113-c297-4f66-92e7-61638e4ccf95
2023-11-27 15:58:32,487 - 
2023-11-27 16:00:35,131 - Order is FULFILLED
2023-11-27 16:00:35,132 - Order is fulfilled successfully! - 283e1113-c297-4f66-92e7-61638e4ccf95
2023

## Access UP42 Storage and get the STAC Collection for downloading

In [64]:
# Initialize the storage and get the list of assets we have
storage = up42.initialize_storage()
assets = storage.get_assets(workspace_id="0775dcad-cdd7-498a-9b17-f74c83f09f76")

2023-11-27 17:12:43,817 - Queried 12 assets for workspace 0775dcad-cdd7-498a-9b17-f74c83f09f76.


'2766510d-24e6-4027-8e44-ccc6d0ff4f87'

### Getting the assets of interest
Since we have some extra datasets, we can turn the assets list into a dataframe so that we may filter for the only six scenes we need which are the most recently created

In [86]:
# Create a dataframe of the assets
assetdf = pd.DataFrame([assets[i].info for i in range(len(assets))])

# Make sure the date is in pandas datetime format
assetdf['createdAt'] = pd.to_datetime(assetdf['createdAt'])

# Sort the dataframe by date descending
sorted_df = assetdf.sort_values(by='createdAt', ascending=False)

# Take the first six rows of the dataframe
last_six_items = sorted_df.head(6)

# Display the results
last_six_items

,id,workspaceId,accountId,createdAt,updatedAt,name,size,contentType,geospatialMetadataExtractionStatus,productId,orderId,producerName,collectionName,tags
0,2766510d-24e6-4027-8e44-ccc6d0ff4f87,0775dcad-cdd7-498a-9b17-f74c83f09f76,0775dcad-cdd7-498a-9b17-f74c83f09f76,2023-11-27 15:34:30.765688+00:00,2023-11-27T15:36:54.961415Z,earthsearch-aws_cfca2863-76f2-4a73-b36d-c3cd7f...,1632688484,application/zip,SUCCESSFUL,c3de9ed8-f6e5-4bb5-a157-f6430ba756da,cfca2863-76f2-4a73-b36d-c3cd7ffd29a9,earthsearch-aws,sentinel-2,[]
1,1beb1fbc-51d7-45ef-98ba-a6bd0799a491,0775dcad-cdd7-498a-9b17-f74c83f09f76,0775dcad-cdd7-498a-9b17-f74c83f09f76,2023-11-27 15:28:16.605240+00:00,2023-11-27T15:30:36.065104Z,earthsearch-aws_de29f367-d76e-40f8-be93-cf2f52...,1619963013,application/zip,SUCCESSFUL,c3de9ed8-f6e5-4bb5-a157-f6430ba756da,de29f367-d76e-40f8-be93-cf2f527ef646,earthsearch-aws,sentinel-2,[]
2,5236a741-b450-4b42-a932-6dbc5c331252,0775dcad-cdd7-498a-9b17-f74c83f09f76,0775dcad-cdd7-498a-9b17-f74c83f09f76,2023-11-27 15:21:27.596164+00:00,2023-11-27T15:23:50.107912Z,earthsearch-aws_2d695dbe-8d29-4348-b789-36ed0d...,1631207277,application/zip,SUCCESSFUL,c3de9ed8-f6e5-4bb5-a157-f6430ba756da,2d695dbe-8d29-4348-b789-36ed0d737eb5,earthsearch-aws,sentinel-2,[]
3,d32f9082-276b-4fad-b310-948819c71fef,0775dcad-cdd7-498a-9b17-f74c83f09f76,0775dcad-cdd7-498a-9b17-f74c83f09f76,2023-11-27 15:14:07.947027+00:00,2023-11-27T15:16:35.004200Z,earthsearch-aws_b9e80fbb-a63b-45bc-b32a-8e3760...,1615499421,application/zip,SUCCESSFUL,c3de9ed8-f6e5-4bb5-a157-f6430ba756da,b9e80fbb-a63b-45bc-b32a-8e37609459d7,earthsearch-aws,sentinel-2,[]
4,effc13f2-9eaa-4843-aaae-f401a9afb21c,0775dcad-cdd7-498a-9b17-f74c83f09f76,0775dcad-cdd7-498a-9b17-f74c83f09f76,2023-11-27 15:05:21.589966+00:00,2023-11-27T15:07:44.359461Z,earthsearch-aws_4c5f6e1b-b725-4890-bb7d-c1c097...,1628152359,application/zip,SUCCESSFUL,c3de9ed8-f6e5-4bb5-a157-f6430ba756da,4c5f6e1b-b725-4890-bb7d-c1c097dec05b,earthsearch-aws,sentinel-2,[]
5,c4dfa664-cdc9-4d0b-8087-62991f4d2138,0775dcad-cdd7-498a-9b17-f74c83f09f76,0775dcad-cdd7-498a-9b17-f74c83f09f76,2023-11-27 14:58:25.291723+00:00,2023-11-27T15:00:45.106630Z,earthsearch-aws_283e1113-c297-4f66-92e7-61638e...,1620308361,application/zip,SUCCESSFUL,c3de9ed8-f6e5-4bb5-a157-f6430ba756da,283e1113-c297-4f66-92e7-61638e4ccf95,earthsearch-aws,sentinel-2,[]


### Downloading the assets

In [98]:
# Create our list of asset ids, their corresponding year and the bands we want (Blue, Green, Red, NIR)
asset_id_list = last_six_items["id"]
year_list = ["2023", "2022", "2021", "2020", "2019", "2018"]
bands_list = ["b02.tiff", "b03.tiff", "b04.tiff", "b08.tiff"]

# Download the assets in the list, the fours bands necessary, and save it in a folder with the corresponding year as the name
for id in range(len(asset_id_list)):
    for band in range(len(bands_list)):
        asset = up42.initialize_asset(asset_id = asset_id_list[id])
        stac_items = asset.stac_items.items
        stac_assets = stac_items[0].assets
        asset.download_stac_asset(
            stac_asset=stac_assets.get(bands_list[band]),
            
            # The data is saved in the Data folder locally and pushed to Github
            output_directory = "/Data/"+year_list[id],
        )

2023-11-27 18:48:10,686 - Initialized Asset(name: earthsearch-aws_cfca2863-76f2-4a73-b36d-c3cd7ffd29a9.zip, asset_id: 2766510d-24e6-4027-8e44-ccc6d0ff4f87, createdAt: 2023-11-27T15:34:30.765688Z, size: 1632688484), contentType: application/zip
2023-11-27 18:48:13,833 - Downloading STAC asset Blue (band 2) - 10m
2023-11-27 18:48:13,835 - Download directory: \Data\2023
197167it [00:24, 8135.02it/s]
2023-11-27 18:48:39,440 - Successfully downloaded the file at \Data\2023\b02.tiff
2023-11-27 18:48:40,068 - Initialized Asset(name: earthsearch-aws_cfca2863-76f2-4a73-b36d-c3cd7ffd29a9.zip, asset_id: 2766510d-24e6-4027-8e44-ccc6d0ff4f87, createdAt: 2023-11-27T15:34:30.765688Z, size: 1632688484), contentType: application/zip
2023-11-27 18:48:43,590 - Downloading STAC asset Green (band 3) - 10m
2023-11-27 18:48:43,592 - Download directory: \Data\2023
210815it [00:27, 7764.40it/s]
2023-11-27 18:49:12,290 - Successfully downloaded the file at \Data\2023\b03.tiff
2023-11-27 18:49:12,948 - Initializ

# Visualizing our data

Now that we have download our data, it is a good idea to check the data to see if everything is what we are expecting before we start the analysis

In [ ]:
import os
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from ipyleaflet import Map, ImageOverlay
from ipywidgets import widgets


def load_tiff_files(folder_path):
    files = {'b02': 'b02.tiff', 'b03': 'b03.tiff', 'b04': 'b04.tiff', 'b08': 'b08.tiff'}
    return {band: rasterio.open(os.path.join(folder_path, filename)) for band, filename in files.items()}

def create_composite(raster_files, true_color=True):
    if true_color:
        # True color composite: B02 (blue), B03 (green), B04 (red)
        return rasterio.plot.get_rgb_array(raster_files['b02'], raster_files['b03'], raster_files['b04'])
    else:
        # False color composite: B03 (blue), B04 (green), B08 (red)
        return rasterio.plot.get_rgb_array(raster_files['b03'], raster_files['b04'], raster_files['b08'])
    
def get_tiff_bounds(tiff_file_path):
    with rasterio.open(tiff_file_path) as src:
        bounds = src.bounds
        return ((bounds.left, bounds.bottom), (bounds.right, bounds.top))
    
year_dropdown = widgets.Dropdown(options=['2018', '2019', '2020', '2021', '2022', '2023'], description='Year:')
color_dropdown = widgets.Dropdown(options=[('True Color', True), ('False Color', False)], description='Mode:')
m = Map(center=aoi_longlat, zoom=20)  # Adjust center and zoom level

def update_map(*args):
    folder_name = year_dropdown.value  # Folder name based on the selected year
    true_color = color_dropdown.value  # True or False color mode

    raster_files = load_tiff_files(folder_path="C:/Data/"+folder_name)
    composite = create_composite(raster_files, true_color)

    # Display the composite on the map as an overlay
    overlay = ImageOverlay(url=composite, bounds=get_tiff_bounds("C:/Data/"+folder_name+"/b02.tiff")) 
    m.clear_layers()
    m.add_layer(overlay)

# Set update_map as the callback for widget changes
year_dropdown.observe(update_map, 'value')
color_dropdown.observe(update_map, 'value')

display(year_dropdown, color_dropdown, m)